In [3]:
import cv2
import numpy as np

In [68]:
#Create background
img = np.full((2560,2560), np.random.randint(230,255),dtype = np.uint8)

nb_speckles = np.random.randint(5e3,7e3)

i=0
while(i<nb_speckles):

    #random ellipse parameters
    center_val = (np.random.randint(0,2560),np.random.randint(0,2560))
    axis_len = (np.random.randint(8,18),np.random.randint(8,18))
    color_val = np.random.randint(0,90)
    
    #draw the ellipse
    cv2.ellipse(img,center_val,axis_len,0,0,360,color_val,-1)
    i += 1

#resize image
img = cv2.resize(img,(0,0),fx=0.1,fy=0.1,interpolation = 3)

#show result
cv2.imshow('test',img)
cv2.waitKey(0);

In [ ]:
#Create displacement field
#Two points are fixed, two are moving
pt_fixed = (1,3)
pt_move = (2,4)

#draw displacement values in pixels
disps = np.random.randn